In [453]:
import pandas as pd
import numpy as np

In [454]:
def price_bond(face_value, coupon_rate, ytm, years_to_maturity, frequency=1):
    coupon_payment = face_value * coupon_rate / frequency
    periods = years_to_maturity * frequency
    discount_rates = (1 + ytm / frequency) ** np.arange(1, periods + 1)
    coupon_payments_pv = np.sum(coupon_payment / discount_rates)
    face_value_pv = face_value / (1 + ytm / frequency) ** periods
    bond_price = coupon_payments_pv + face_value_pv
    return bond_price

def calculate_duration(face_value, coupon_rate, ytm, years_to_maturity, frequency=1, modified=True):
    bond_price = price_bond(
        face_value=face_value,
        coupon_rate=coupon_rate,
        ytm=ytm,
        years_to_maturity=years_to_maturity,
        frequency=frequency
    )
    total_time_periods = years_to_maturity * frequency
    time_periods = np.arange(1, total_time_periods + 1)
    discount_factors = (1 + ytm / frequency) ** time_periods  # Correct discount rate calculation
    coupon_payment = face_value * coupon_rate / frequency
    cash_flows = np.full(total_time_periods, coupon_payment)
    cash_flows[-1] += face_value  # Add the face value to the last payment

    # Correctly discount and weight the cash flows
    discounted_cash_flows = cash_flows / discount_factors
    weighted_discounted_cash_flows = discounted_cash_flows * time_periods
    duration = np.sum(weighted_discounted_cash_flows) / bond_price

    if modified:
        duration = duration / (1 + ytm / frequency)

    return duration

In [455]:
YIELD_CURVE_SHIFT_FACTOR = 0.01
SHIFT_PERCENT = True
FLATTEN_YIELD_CURVE = False
FIRST_PERIOD_RATE = 0.02
MAX_MATURITY = 30
MATURITIES = np.arange(1, MAX_MATURITY + 1, 1)
TOTAL_YEARS = 30
TOTAL_YEARS_ARR = np.arange(0, TOTAL_YEARS + 1, 1)

BOND_MATURITY = 10
FACE_VALUE = 1_000
COUPON = 0.1
COUPON_GROWTH_RATE = 0
FREQUENCY = 1

# Yield Curve

In [456]:
yield_curve_shift_arr = TOTAL_YEARS_ARR * YIELD_CURVE_SHIFT_FACTOR


yield_curve_shape_arr = np.log10(MATURITIES, ) if not FLATTEN_YIELD_CURVE else 0
yield_curve_df = pd.DataFrame(data=FIRST_PERIOD_RATE, index=MATURITIES, columns=TOTAL_YEARS_ARR)
yield_curve_df = yield_curve_df.multiply(yield_curve_shape_arr + 1, axis=0)

if SHIFT_PERCENT:
    yield_curve_df = yield_curve_df.multiply(yield_curve_shift_arr + 1, axis=1)
else:
    yield_curve_df = yield_curve_df.add(yield_curve_shift_arr, axis=1)

In [457]:
yield_curve_df.plot(legend=False)

In [458]:
bond_names = [f'Bond {x}' for x in range(1, BOND_MATURITY + 1)]
bond_default_df = pd.DataFrame(index=TOTAL_YEARS_ARR, columns=bond_names)

# Maturities

In [459]:
maturity_pattern_arr = np.arange(BOND_MATURITY, 0, -1)

# Use broadcasting to fill in the matrix
indices = np.arange(TOTAL_YEARS + 1)[:, None] + np.arange(BOND_MATURITY)
wrapped_indices = np.mod(indices, BOND_MATURITY)

# Fill the matrix using the wrapped indices to repeat the maturity pattern
maturity_matrix = maturity_pattern_arr[wrapped_indices]

maturity_df = bond_default_df.copy()
maturity_df.loc[:, :] = maturity_matrix

# Prices

In [460]:
coupon_df = bond_default_df.copy()
coupon_df.loc[:, :] = COUPON * FACE_VALUE
price_df = bond_default_df.copy()
face_value_df = bond_default_df.copy()
face_value_df.loc[:, :] = 0
repurchase_df = bond_default_df.copy()
repurchase_df.loc[:, :] = 0

for index in maturity_df.index:
    for col in maturity_df.columns:
        maturity = maturity_df.loc[index, col]
        price = price_bond(
            face_value=FACE_VALUE,
            coupon_rate=COUPON,
            ytm=yield_curve_df.loc[BOND_MATURITY, index],
            years_to_maturity=maturity,
            frequency=FREQUENCY
        )
        price_df.loc[index, col] = price
        
        if maturity == BOND_MATURITY:
            face_value_df.loc[index, col] = FACE_VALUE
            repurchase_df.loc[index, col] = price

coupon_df = coupon_df.iloc[1:, :]
face_value_df = face_value_df.iloc[1:, :]
repurchase_df = repurchase_df.iloc[1:, :]

In [461]:
price_dollar_return_df = (price_df - price_df.shift()).dropna()
roll_dollar_return = face_value_df - repurchase_df

In [462]:
total_dollar_return = price_dollar_return_df + roll_dollar_return + coupon_df

In [463]:
price_dollar_return_df.sum(axis=1)

In [464]:
roll_dollar_return.sum(axis=1)

In [465]:
total_dollar_return

In [466]:
total_dollar_return.sum(axis=1) / price_df.iloc[:-1, :].sum(axis=1).values

In [467]:
price_df